In [1]:
%run RaceCar.py
import numpy as np
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generate_map(w_cells, h_cells, min_lim, max_lim):
    race_map = np.zeros((w_cells, h_cells), dtype=np.int32)
    # Set boundaries
    race_map[w_cells // 2:, h_cells // 2:] = -1
    lims = np.random.randint(min_lim, max_lim, size=4)
    for h in range(h_cells):
        lims[0] = np.amax([min_lim, np.amin([max_lim, lims[0] + np.random.choice([-1, 0, 1])])])
        race_map[h, :lims[0]] = -1
        if h > h_cells // 2:
            lims[1] = np.amax([min_lim, np.amin([max_lim, lims[1] + np.random.choice([-1, 0, 1])])])
            race_map[h, h_cells // 2 - lims[1]:] = -1
    for w in range(w_cells):
        lims[2] = np.amax([min_lim, np.amin([max_lim, lims[2] + np.random.choice([-1, 0, 1])])])
        race_map[:lims[2], w] = -1
        if w > w_cells // 2:
            lims[3] = np.amax([min_lim, np.amin([max_lim, lims[3] + np.random.choice([-1, 0, 1])])])
            race_map[w_cells // 2 - lims[3]:, w] = -1
    # Set start line
    race_map[-1, np.argwhere(race_map[-1, :] == 0)] = 1
    # Set finish line
    race_map[np.argwhere(race_map[:, -1] == 0), -1] = 2
    return race_map

#
width = 850
height = 850
w_cells = 30
h_cells = 30
min_lim = 1
max_lim = 6

race_map = generate_map(w_cells, h_cells, min_lim, max_lim)


def func():
    while True:
        draw_map(screen, race_map)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        pygame.display.update()

In [3]:
# Number of states.
S = race_map.shape[0] * race_map.shape[1]
init_states = np.argwhere(race_map == 1)
terminal_states = np.argwhere(race_map == 2)
print(init_states)
# Number of actions (considering only "positive" movements).
actions_x = 6
actions_y = 6
A = actions_x * actions_y
# Init a randomic policy.
pi = np.random.randint(A, size=S)
# Discount factor.
gamma = 1.0
# Exploration degree.
epsilon = 0.1
# Number of episodes.
episodes = 10000
# Inizialization.
N = np.zeros((S, A), dtype=np.int32)
Q = np.random.normal(loc=10.0, size=(S, A))
for T in terminal_states:
    Q[np.ravel_multi_index((T[0], T[1]), (race_map.shape[0], race_map.shape[1])), :] = 0.0

# Loop on episodes.
for e in range(episodes):
    print("NEW: {}".format(e)) #! DEBUG
    # Exploring start.
    init_coords = np.copy(init_states[np.random.randint(init_states.shape[0])])
    #print(init_coords) #! DEBUG
    # Run a game.
    car = RaceCar(race_map, init_coords, pi, epsilon, width, height, actions_x, actions_y)
    disp = False
    if (e > 9950):
        disp = True
    car.run(disp)
    states = car.get_states()
    actions = car.get_actions()
    rewards = car.get_rewards()
    G = 0.0
    for t, s_t in reversed(list(enumerate(states))):
        G = rewards[t] + gamma * G
        N[s_t, actions[t]] += 1
        Q[s_t, actions[t]] += (1.0 / N[s_t, actions[t]]) * (G - Q[s_t, actions[t]])
        pi[s_t] = np.argmax(Q[s_t, :])

[[29  6]
 [29  7]
 [29  8]
 [29  9]]
NEW: 0
NEW: 1
NEW: 2
NEW: 3
NEW: 4
NEW: 5
NEW: 6
NEW: 7
NEW: 8
NEW: 9
